In [1]:
# Install Dependencies
!pip install -qU datasets pinecone-client sentence-transformers torch pyngrok

In [1]:
# Import modules
from flask import Flask, request, jsonify, make_response
import threading
from werkzeug.serving import make_server
from pyngrok import ngrok

# Create App
app = Flask(__name__)

# Running Flask in a separate thread
def run_app():
    global server
    server = make_server('127.0.0.1', 5000, app)
    server.serve_forever()



In [2]:
# Define Routes
@app.route("/")
def home():
    return "Api works!"

@app.route("/test_route")
def test_route():
    return jsonify("test")


In [ ]:
#Connect to vector DB
import pinecone
from config import PINECONE_API_KEY

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment="gcp-starter"
)


In [6]:
# launch server
flask_thread = threading.Thread(target=run_app)
flask_thread.start()

# Open an ngrok tunnel to the HTTP server
public_url = ngrok.connect(5000)
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:5000\"".format(public_url))


 * ngrok tunnel "NgrokTunnel: "https://02f1-37-120-244-62.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"


127.0.0.1 - - [30/Sep/2023 23:12:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2023 23:12:32] "GET /favicon.ico HTTP/1.1" 404 -


In [5]:
# Shutdown Api
if server:
    server.shutdown()
# Disconnect the ngrok tunnel
ngrok.disconnect(public_url)
